In [66]:
import boto3
import networkx as nx
import numpy as np
import uuid

import os
import configparser

In [67]:
config = configparser.ConfigParser()

# Write example config to disk.
config['AWS'] = {
    'ACCESS_KEY': '',
    'SECRET_KEY': '',
    'SESSION_TOKEN': ''
}

config['Master'] = {
    'API_KEY': ''
}

with open('config.txt.sample', 'w') as configfile:
    config.write(configfile)
    
config = configparser.ConfigParser()
config.read('config.txt')

session = boto3.Session(
    aws_access_key_id=config['AWS']['access_key'], 
    aws_secret_access_key=config['AWS']['secret_key'], 
)


In [93]:
def getInstanceIdsByType():
    response = session.client("ec2").describe_instances()
    reservations = response['Reservations']
    
    instancesByType = dict()
    for reservation in reservations:
        for instance in reservation['Instances']:
            for tag in instance['Tags']:
                if tag['Key'] == 'instance_type':
                    instancetype = tag['Value']
                    if not instancetype in instancesByType:
                        instancesByType[instancetype] = [instance['InstanceId']]
                    else:
                        instancesByType[instancetype].append(instance['InstanceId'])
    
    return instancesByType    

def terminateInstancesWithIds(ids):
    session.client("ec2").terminate_instances(InstanceIds=ids)

def createMaster(session):
    ec2_client = session.client('ec2')

def generate_graph_file(directory, n, p, prepend_vertex_weights=False):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    file = os.path.join(directory, '%s-%d-%s.graph' % (uuid.uuid4(), n, p))

    # Generate ER graph with n nodes and edge probability p
    G = nx.erdos_renyi_graph(int(n), p)
    node_weights = []

    # Generate random edge weights.
    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = round(np.random.random(), 2)

    nx.write_weighted_edgelist(G, file, delimiter=",")

    if prepend_vertex_weights:
        # Prepend weights
        for u in G.nodes():
            node_weights.append(str(1.0 / n)) # Uniform for pagerank.

        with open(file, 'r') as original: data = original.read()
        with open(file, 'w') as modified: modified.write(",".join(node_weights) + "\n" + data)
            
    return file

def run_experiment():
    # Connecto to EC2
    ec2_client = session.client('ec2')
    
    # Spin up a master.
    response = ec2_client.run_instances(LaunchTemplate={'LaunchTemplateName': "Master"}, MinCount=1, MaxCount=1)

    # Terminate all masters.
    instances = getInstanceIdsByType()
    terminateInstancesWithIds(instances['master'])

In [97]:
#run_experiment()


In [46]:
# directory = 'graphs'
# node = [1e1, 1e2, 1e3]
# p_edge = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# # For each node_vertex combination
# for n in node:
#     for p in p_edge:
#         generate_graph_file(directory, n, p, prepend_vertex_weights=True)